## [CSI Process Workflow] Global Shelf [Generating the Depth-Class Digital Surface Models]
<br>
Notebook scope:

- Create the GLOBAL Geomorphon-based shelf models for the shallow, intermediate, deep, and para shelf classes
- Create the GLOBAL Bathymetry-based shelf models for the shallow, intermediate, deep, and para shelf classes
- Create the GLOBAL Slope-based shelf models for the shallow, intermediate, deep, and para shelf classes
- Create the ARCTIC and ANTARCTIC the Geomorphon, Bathymetry, and Slope shelf models for the shallow, intermediate, deep, and para shelf classes


**NOTE: we assume here that the (non-classified) geomorphon-forms, bathymetry and slope DSMs already exist for the GLOBAL, ARCTIC, and ANTARCTIC regions!**

Notebook details processes to generate depth class surface models for the global (including the poles) geomorphon shelf model. The model is defined using the geomorphon 'flat' form where 'flat' is defined as slopes that do not exceed 0.69°. Further, depths are constrained (based on exploratory clustering) to four depth windows: 

#### The Shelf Depth Classes:

- (shallow)  0 meters to -166.8 meters 
- (intermediate) -168.3 meters to -420.2 meters 
- (deep) -422.9 meters to -742.9 meters 
- (para-shelf, shelf-like) -742.9 meters down to -1,211 meters

##### Notes:
- All data here is projected in the interrupted Goode Homolosine Projection (the IGH is an Equal Area projection)
- flatness = 0° to 0.69°
- search=5  (maximum window size-so that we don't lose too much of the small shelf bits)
- distance=0
- extract from the 1km resolution ETOPO1 topobathy DEM
- Region Extents: xmin=-20015000, ymin=-8673000,xmax= 20015000, ymax=8673000
- can use the ETOPO1_World_Shelf_Zone(k) layers, where k is the zone number from 1 thru 13, as masks for analysis

##### The k=13 Zones:
|1   | Eastern North America <br> 
|2	 | Western North America <br>
|3	 | Eastern South America <br>
|4	 | Western South America <br>
|5	 | Europe <br>
|6	 | Western Africa <br>
|7	 | Indian/Oceana <br>
|8	 | Eastern Asia <br>
|9	 | Australia <br>
|10	 | Arctic <br>
|11	 | Antarctic <br>
|12	 | Mediterranean <br> 
|13	 | New Zealand (not currently included in the analysis)

#### Declare the class definition used to initialized the GRASS 7.1 instances needed for the processing to come (an instance allows calling GRASS libraries from outside of a GRASS session--cool eh?)

In [1]:
# This class definition is used to initialize an instance of GRASS 7.x in this domain space
class GrassBASE:
    def initGRASSEnv( self, database, loc, map ):
        gisbase=os.environ[ 'GISBASE' ] = '/Applications/GRASS-7.1.app/Contents/MacOS'
        gisdbase=database
        location=loc
        mapset=map
        sys.path.append(os.path.join(os.environ['GISBASE'], "etc", "python"))
        import grass.script as gscript
        import grass.script.setup as gsetup
        gsetup.init( gisbase, gisdbase, location, mapset )
        print(gscript.gisenv())
        print('GRASS Application base path:',gisbase)

### WORKFLOW for creating the *GLOBAL* shelf depth-class DSMs (does not include the Arctic and Anarctic regions--see below for generating data for these polar regions)

1.) [in GRASS] Extract global depth classes from the World bathymetry and slope data

- ETOPO1_World_Shelf_bathy_1km
- ETOPO1_World_Shelf_slope_1km

#### 1.) Instantiate a new GRASS 7.1 session

In [2]:
import os, sys

# set the location and mapset for the GRASS instance to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'csi_shelf_mapping_global'                              # name of location
mapset = 'user'                                                    # name of mapset containing data


# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'csi_shelf_mapping_global'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib


### Generate the four shelf class raster models for the GLOBAL data: 
Note that the GLOBAL data do not include the polar Arctic and Anarctic regions! See further along in the notebook for coverage of these latter (chilly) zones...

In [3]:
import grass.script as gscript

##### Generate raster models for the shallow shelf class (z:0 to -166.8 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_World_Shelf_shallow_1km=if(ETOPO1_bathy_1km >= -166.8 && ETOPO1_Slope_1km <= 0.69 &&\
ETOPO1_geomorph_forms_1km==1, ETOPO1_geomorph_forms_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_shallow_bathy_1km=if(ETOPO1_bathy_1km >= -166.8 && ETOPO1_Slope_1km <= 0.69 && \
ETOPO1_geomorph_forms_1km==1, ETOPO1_bathy_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_shallow_slope_1km=if(ETOPO1_bathy_1km >= -166.8 && ETOPO1_Slope_1km <= 0.69 && \
ETOPO1_geomorph_forms_1km==1, ETOPO1_Slope_1km,  null() )' )


##### Generate raster models for the intermediate shelf class (z:-168 to -420 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_World_Shelf_intermed_1km=if(ETOPO1_bathy_1km >= -420.2 && ETOPO1_bathy_1km <= -168.3 && \
ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_geomorph_forms_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_intermed_bathy_1km=if(ETOPO1_bathy_1km >= -420.2 && ETOPO1_bathy_1km <= -168.3 &&\
ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_bathy_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_intermed_slope_1km=if(ETOPO1_bathy_1km >= -420.2 && ETOPO1_bathy_1km <= -168.3 &&\
ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_Slope_1km,  null() )' )


##### Generate raster models for the deep shelf class (z:-423 to -743 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_World_Shelf_deep_1km=if(ETOPO1_bathy_1km >= -742.9 && ETOPO1_bathy_1km <= -422.9 && \
ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_geomorph_forms_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_deep_bathy_1km=if(ETOPO1_bathy_1km >= -742.9 && ETOPO1_bathy_1km <= -422.9 && \
ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_bathy_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_deep_slope_1km=if(ETOPO1_bathy_1km >= -742.9 && ETOPO1_bathy_1km <= -422.9 && \
ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_Slope_1km,  null() )' )


##### Generate raster models for the para-shelf class (z:-743 to -1211 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_World_Shelf_parashelf_1km=if(ETOPO1_bathy_1km >= -1211.0 && ETOPO1_bathy_1km <= -742.9 && \
ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_geomorph_forms_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_parashelf_bathy_1km=if(ETOPO1_bathy_1km >= -1211.0 && ETOPO1_bathy_1km <= -742.9 \
&& ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_bathy_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_World_Shelf_parashelf_slope_1km=if(ETOPO1_bathy_1km >= -1211.0 && ETOPO1_bathy_1km <= -742.9 \
&& ETOPO1_Slope_1km <= 0.69 && ETOPO1_geomorph_forms_1km==1, ETOPO1_Slope_1km,  null() )' )

### WORKFLOW for creating the *POLAR* shelf depth-class DSMs (includes the Arctic and Antarctic regions)

[in GRASS] Extract Arctic and Antarctic depth classes from their respective bathymetry and slope data

- ETOPO1_Arctic_Shelf_bathy_1km
- ETOPO1_Arctic_Shelf_slope_1km
- ETOPO1_Antarctic_Shelf_bathy_1km
- ETOPO1_Antarctic_Shelf_slope_1km

### The ARCTIC:

##### Instantiate new GRASS instance for processing in the Arctic

In [12]:
# set the location and mapset for the Arctic GRASS instance to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'CSM_Arctic_Region'                                     # name of location
mapset = 'user'                                                    # name of mapset containing data


# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


# ## set the GRASS working region (for the Arctic), too:
gscript.run_command('g.region', flags='p', region='ETOPO1_arctic_1km')


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'CSM_Arctic_Region'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib


0

### Generate the four shelf class raster models for the ARCTIC: 

In [13]:
import grass.script as gscript

# ## Generate raster models for the shallow shelf class (z:0 to -166.8 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Arctic_Shelf_shallow_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -166.8 && \
ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && ETOPO1_geomorph_arctic_forms_1km==1, \
ETOPO1_geomorph_arctic_forms_1km,  null() )', overwrite=True )

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_shallow_bathy_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -166.8 && \
ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && ETOPO1_geomorph_arctic_forms_1km==1, \
ETOPO1_Arctic_Shelf_bathy_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_shallow_slope_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -166.8 && \
ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && ETOPO1_geomorph_arctic_forms_1km==1, \
ETOPO1_Arctic_Shelf_slope_1km,  null() )' )


# ## Generate raster models for the intermediate shelf class (z:-168 to -420 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Arctic_Shelf_intermed_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -420 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -168 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_geomorph_arctic_forms_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_intermed_bathy_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -420 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -168 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_Arctic_Shelf_bathy_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_intermed_slope_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -420 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -168 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_Arctic_Shelf_slope_1km,  null() )' )


# ## Generate raster models for the deep shelf class (z:-423 to -743 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Arctic_Shelf_deep_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -743 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -423 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_geomorph_arctic_forms_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_deep_bathy_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -743 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -423 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_Arctic_Shelf_bathy_1km,  null() )' )

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_deep_slope_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -743 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -423 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_Arctic_Shelf_slope_1km,  null() )' )


# ## Generate raster models for the para-shelf class (z:-743 to -1211 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Arctic_Shelf_parashelf_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -1211 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -743 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_geomorph_arctic_forms_1km,  null() )' ) 

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_parashelf_bathy_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -1211 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -743 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_Arctic_Shelf_bathy_1km,  null() )' ) 

gscript.mapcalc( 'ETOPO1_Arctic_Shelf_parashelf_slope_1km=if(ETOPO1_Arctic_Shelf_bathy_1km >= -1211 && \
ETOPO1_Arctic_Shelf_bathy_1km <= -743 && ETOPO1_Arctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_geomorph_arctic_forms_1km==1, ETOPO1_Arctic_Shelf_slope_1km,  null() )' ) 

### The ANTARCTIC:

##### Instantiate new GRASS instance for processing in the ANTARCTIC:

In [14]:
# set the location and mapset for the Antarctic GRASS instance to be created
gisdbase = '/Users/paulp/Documents/csi/cshelves/data/GRASSData'    # location of GRASS database  
location = 'CSM_Antarctic_Region'                                     # name of location
mapset = 'user'                                                    # name of mapset containing data


# initiate a GRASS instance/environment

print('')
print('********************************************')
print('Setting up a GRASS 7.1 Environment') 
g = GrassBASE()
g.initGRASSEnv( gisdbase, location, mapset )
#print(sys.path)
print()
print(os.environ[ 'DYLD_LIBRARY_PATH' ])


# ## set the GRASS working region (for the Arctic), too:
gscript.run_command('g.region', flags='p', region='ETOPO1_antarctic_1km')


********************************************
Setting up a GRASS 7.1 Environment
{u'MAPSET': u'user', u'GISDBASE': u'/Users/paulp/Documents/csi/cshelves/data/GRASSData', u'LOCATION_NAME': u'CSM_Antarctic_Region'}
('GRASS Application base path:', '/Applications/GRASS-7.1.app/Contents/MacOS')
()
:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib:/Applications/GRASS-7.1.app/Contents/MacOS/lib


0

### Generate the four shelf class raster models for the ANTARCTIC: 

In [16]:
import grass.script as gscript

# ## Generate raster models for the shallow shelf class (z:0 to -166.8 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_shallow_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -166.8 && \
ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && ETOPO1_Antarctic_Geomorph_Forms_1km==1, \
ETOPO1_Antarctic_Geomorph_Forms_1km,  null() )', overwrite=True )

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_shallow_bathy_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -166.8 && \
ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && ETOPO1_Antarctic_Geomorph_Forms_1km==1, \
ETOPO1_Antarctic_Shelf_bathy_1km,  null() )', overwrite=True )

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_shallow_slope_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -166.8 && \
ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && ETOPO1_Antarctic_Geomorph_Forms_1km==1, \
ETOPO1_Antarctic_Shelf_slope_1km,  null() )', overwrite=True )


# ## Generate raster models for the intermediate shelf class (z:-168 to -420 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_intermed_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -420 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -168 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Geomorph_Forms_1km,  null() )', overwrite=True )

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_intermed_bathy_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -420 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -168 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Shelf_bathy_1km,  null() )', overwrite=True)

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_intermed_slope_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -420 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -168 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Shelf_slope_1km,  null() )', overwrite=True)


# ## Generate raster models for the deep shelf class (z:-423 to -743 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_deep_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -743 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -423 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Geomorph_Forms_1km,  null() )', overwrite=True )

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_deep_bathy_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -743 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -423 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Shelf_bathy_1km,  null() )', overwrite=True)

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_deep_slope_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -743 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -423 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Shelf_slope_1km,  null() )', overwrite=True)


# ## Generate raster models for the para-shelf class (z:-743 to -1211 meters;  m:0 to 0.69°;  form=1):
gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_parashelf_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -1211 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -743 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Geomorph_Forms_1km,  null() )', overwrite=True ) 

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_parashelf_bathy_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -1211 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -743 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Shelf_bathy_1km,  null() )', overwrite=True ) 

gscript.mapcalc( 'ETOPO1_Antarctic_Shelf_parashelf_slope_1km=if(ETOPO1_Antarctic_Shelf_bathy_1km >= -1211 && \
ETOPO1_Antarctic_Shelf_bathy_1km <= -743 && ETOPO1_Antarctic_Shelf_slope_1km <= 0.69 && \
ETOPO1_Antarctic_Geomorph_Forms_1km==1, ETOPO1_Antarctic_Shelf_slope_1km,  null() )', overwrite=True ) 